In [1]:
import sys
import os
import torch
import glob
import pandas as pd
import numpy as np
from sklearn import metrics,preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
import collections

import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data_utils

# Imports for PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#constant declaration
batch_size = 30
num_classes  = 2
epochs = 3
learning_rate = 1e-3

In [2]:
# For full dataset, run tran_feature_selection notebook before this one
# For small subset of data, run tran_sample_preprocessing notebook beofore this one (ideal for testing model on CPU)

%store -r benign_flows  
%store -r mixed_flows
%store -r features

dim = len(features)


normalizer = preprocessing.Normalizer(norm="l2")
normalized_train = normalizer.fit_transform(benign_flows[features]) #axis?
train_X = pd.DataFrame(normalized_train, columns = features)  #dataframe

normalized_test = normalizer.transform(mixed_flows[features])
test_X = pd.DataFrame(normalized_test, columns = features)
test_y = mixed_flows.is_attack

In [3]:
x = benign_flows.loc[:, features].values
x = StandardScaler().fit_transform(x) # normalizing the features
np.mean(x),np.std(x)

/users/grad/ifath/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)


numpy.ndarray

In [ ]:
# finding the top k components whose cumulative variance exceeds 95%
  
pca = PCA(0.95)
pca.fit(x)
top_k_component = pca.n_components_
top_k_component


In [ ]:
# re-run PCA with only top k features again
top_k_features = features[:top_k_component]

top_k_x = benign_flows.loc[:, top_k_features].values
top_k_x = StandardScaler().fit_transform(top_k_x) # normalizing the features
np.mean(top_k_x),np.std(top_k_x)

top_k_pca = PCA(n_components=top_k_component-1)   
top_k_principalComponents = top_k_pca.fit_transform(top_k_x)

top_k_var=np.cumsum(np.round(top_k_pca.explained_variance_ratio_, decimals=3)*100)
# print(top_k_var)


In [ ]:
# inverse mapping of low dimensional to original coordinate space
data_reduced = np.dot(x-pca.mean_, pca.components_.T) # transform
data_original = np.dot(data_reduced, pca.components_)+ pca.mean_ # inverse_transform
re_error = data_original-x  #reconstruction error
print(re_error.shape)
print(re_error.shape[1])

In [ ]:
labels = ['is_attack']

x_train = x

benign_flows['is_attack']= 0
y_train = benign_flows.loc[:, labels].values

x_test = mixed_flows.loc[:, features].values
y_test = mixed_flows.loc[:, labels].values    

# normalizing
#x_train = preprocessing.normalize(x_train, axis=0)
x_test = preprocessing.normalize(x_train, axis=0)

In [ ]:
# Calculate reconstruction loss for train partition (benign flow data)
from numpy import linalg as LA

start=2
error_record=[]
for i in range(start,top_k_component):
    pca = PCA(n_components=i)
    pca2_results = pca.fit_transform(x_train)
    pca2_proj_back=pca.inverse_transform(pca2_results)
    total_loss=LA.norm((x_train-pca2_proj_back),None)
    error_record.append(total_loss)

In [ ]:
# plt.clf()
# plt.figure(figsize=(15,15))
# plt.title("reconstruction error of pca")
# plt.xlabel("components")
# plt.ylabel("loss value ")
# plt.plot(error_record,'r')
# plt.xticks(range(len(error_record)), range(start,top_k_component), rotation='vertical')
# plt.xlim([-1, len(error_record)])
# # plt.show()

In [ ]:
# Calculate reconstruction loss for test partition (mixed flow data)

start=2
test_loss_pca=[]

for i in range(start,dim):
    pca = PCA(n_components=i)
    pca2_results = pca.fit_transform(x_test)
    pca2_proj_back=pca.inverse_transform(pca2_results)
    total_loss=LA.norm((x_test-pca2_proj_back),None)
    test_loss_pca.append(total_loss)

In [ ]:
pca = PCA(0.95)
train_pca = pca.fit_transform(x_train)
test_pca = pca.transform(x_test)

In [ ]:
# necessary libraries to run the deep learning model
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.optimizers import RMSprop

In [ ]:
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(top_k_component,)))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# model.summary()

In [ ]:
# compiling and training the model with the reduced dimension
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(train_pca, y_train,batch_size=batch_size,epochs=epochs,verbose=1,
                    validation_data=(test_pca, y_test))

In [ ]:
# checking how the model works with all the features
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(dim,)))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,
                    validation_data=(x_test, y_test))

In [ ]:
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
classifier.fit(train_pca, y_train)

y_pred = classifier.predict(test_pca)
y_pred

In [ ]:
# Performance Evaluation
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score
from sklearn.metrics import accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)
y_test = 1*y_test
z =  accuracy_score(y_test, y_pred)
z

In [ ]:
# #Visualize all the original dimensions
# import plotly.express as px

# pca = PCA(n_components=top_k_component)
# components = pca.fit_transform(train_X)

# total_var = pca.explained_variance_ratio_.sum() * 100

# labels = {str(i): f"PC {i+1}" for i in range(top_k_component)}
# labels['color'] = 'flowInd'

# fig = px.scatter_matrix(
#     components,
#     color=mixed_flows['is_attack'],
#     dimensions=range(top_k_component),
#     labels=labels,
#     title=f'Total Explained Variance: {total_var:.2f}%',
# )
# fig.update_traces(diagonal_visible=False)
# fig.show()

In [ ]:
import plotly.express as px
pca = PCA()
pca.fit(train_X)

exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
print(exp_var_cumul)

px.area(
    x=range(1, exp_var_cumul.shape[0]+1),
    y=exp_var_cumul,
    labels={"x": "# Components", "y": "Explained Variance"}
)